In [95]:
import pandas as pd                         
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random

In [96]:
df = pd.read_csv("data/Office_Products_5.csv")             
data = df.to_numpy()

In [97]:
data = data[int(0.85*len(data)):]   #test data 

In [102]:
tr4 = []
tr5 = []
tr_other = []
for d in data:
    if d[2] == 4:
        tr4.append(d)
    elif d[2] == 5:
        tr5.append(d)
    else:
        tr_other.append(d)
data = tr_other + tr4[:10000] + tr5[:10000]
random.shuffle(data)

In [103]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in data:
    user,item = d[0], d[1]
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

## Simple (bias only) latent factor-based recommender

In [104]:
N = len(data)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())
alpha = sum([d[2] for d in data]) / len(data)
labels = [d[2] for d in data]

In [105]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [106]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [107]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [108]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))


In [109]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in data]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [110]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(data)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in data:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - d[2]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

In [111]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 0.001))

MSE = 1.7930627363636462
MSE = 1.7349223215284646
MSE = 2.8043078245408615
MSE = 1.6795845118472694
MSE = 1.5497686449877441
MSE = 1.5475017971749363
MSE = 1.5390552507880895
MSE = 1.4439219234602276
MSE = 1.394225783119682
MSE = 1.3564022816178827
MSE = 1.3392963700596485
MSE = 1.317684169980549
MSE = 1.3042517158704727
MSE = 1.3026225885281106
MSE = 1.3068176757042658
MSE = 1.3082436620334887
MSE = 1.3086929797091735
MSE = 1.2942679736134057
MSE = 1.3030153039338257
MSE = 1.298763286331498
MSE = 1.2933768873843943
MSE = 1.2923834189095926
MSE = 1.2923129424992428
MSE = 1.2681034276943923
MSE = 1.2867813928433567
MSE = 1.2898642401473837
MSE = 1.291681075360293
MSE = 1.2905765693943716
MSE = 1.2911661154543488
MSE = 1.290032050616978
MSE = 1.2892082700428822
MSE = 1.2885779673008948
MSE = 1.2887383123039797
MSE = 1.2889769333740826
MSE = 1.2895425509427825
MSE = 1.289868230942792
MSE = 1.290326248940216
MSE = 1.2891667568349836
MSE = 1.2898566288695485
MSE = 1.2893411782978146
MSE = 1

(array([ 3.31651145,  0.03931447,  0.12689249, ...,  0.01819081,
        -0.06343828, -0.00841552]),
 1.4641853485516134,
 {'grad': array([ 2.78783493e-05, -3.01187491e-08,  8.13465010e-08, ...,
          6.57635202e-10,  7.30715632e-09, -1.22168613e-08]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 57,
  'nit': 48,
  'warnflag': 0})

## Complete latent factor model 

In [112]:
alpha =  sum([d[2] for d in data]) / len(data)
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
userGamma = {}
itemGamma = {}

In [113]:
K = 2

In [114]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [115]:
def unpack(theta):   
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

In [116]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [117]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [118]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in data]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [119]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(data)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for d in data:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - d[2]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

In [120]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, 0.001))

MSE = 1.7930571430077034
MSE = 1.7364792710173
MSE = 2.957254259880826
MSE = 1.680402728883922
MSE = 1.546150641628824
MSE = 1.5436958292693135
MSE = 1.5346203890469674
MSE = 1.4379653375559938
MSE = 1.3859016810917637
MSE = 1.347333912529546
MSE = 1.3284358712248272
MSE = 1.3073193934523057
MSE = 1.2914025914799037
MSE = 1.2858645925486962
MSE = 1.2911830358409555
MSE = 1.2927951195390375
MSE = 1.2945195074301643
MSE = 1.2861204287069032
MSE = 1.28636510062141
MSE = 1.2848532207337542
MSE = 1.286828404565396
MSE = 1.2869630263348386
MSE = 1.287491163044251
MSE = 1.2878688726083856
MSE = 1.286479234741703
MSE = 1.288922286201256
MSE = 1.2838571772120904
MSE = 1.2873797150097925
MSE = 1.287426865457238
MSE = 1.2878106102265485
MSE = 1.2912491200561038
MSE = 1.2885151252848364
MSE = 1.2890452902623828
MSE = 1.289348313798618
MSE = 1.289449803778239
MSE = 1.2892714317173612
MSE = 1.2890892097628135
MSE = 1.288690705736059
MSE = 1.2885423052228755
MSE = 1.2886899716951263
MSE = 1.288756845

(array([ 3.31649159e+00,  3.93308294e-02,  1.26860309e-01, ...,
        -3.13177909e-06, -2.58409220e-06, -4.40286665e-06]),
 1.46418533145119,
 {'grad': array([-1.40695690e-05,  6.58910615e-09,  6.99511556e-09, ...,
         -5.94065657e-09, -5.20319882e-09, -8.77115841e-09]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 72,
  'nit': 61,
  'warnflag': 0})